In [1]:
import os

In [2]:
!mkdir /content/dataset/

In [3]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 28.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.0 MB/s eta 0:00

In [4]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import string
import re
import time

In [5]:
import json
import os
from pathlib import Path
from typing import *

import pandas as pd 
import scipy
from datasets import ClassLabel, Dataset
from sklearn import metrics
from sklearn.model_selection import train_test_split
import transformers
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
    Trainer,
    TrainingArguments,
)
print(transformers.__version__)

4.28.1


In [6]:
%cd /content/drive/MyDrive/CS5246Project/stock_dataset/
!cp Combined_train.csv /content/dataset/
!cp Combined_valid.csv /content/dataset/
!cp Combined_test.csv /content/dataset/

df_train = pd.read_csv('/content/dataset/Combined_train.csv')
df_valid = pd.read_csv('/content/dataset/Combined_valid.csv')
df_test = pd.read_csv('/content/dataset/Combined_test.csv')

/content/drive/MyDrive/CS5246Project/stock_dataset


In [7]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

In [8]:
def clean_text(text):
    text = str(text).lower()    # Making Text Lowercase
    text = re.sub('b\"|b\'|\\\\|\\\"', '', text) # remove the format that text starts with b" or b'
    text = re.sub('\[.*?\]', '', text) # remove text in [] brackets
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)
    return text

def clean_contractions(text, mapping):
    # Clean contraction by using contraction mapping   
    for word in mapping.keys():
        if str(word) in text:
            text = text.replace(str(word), str(mapping[word]))
    #Remove Punctuations
    text = re.sub('[^a-zA-Z]', ' ', text)
 
    return text

def remove_space(text): 
    ## remove space that are too long
    text = text.strip()
    text = text.split()
    return " ".join(text)

def text_preprocessing_pipeline(text):
    text = clean_text(text)
    text = clean_contractions(text, contraction_mapping)
    text = remove_space(text)
    return text

In [9]:
for i in range(df_train.shape[0]):
    for j in range(2, df_train.shape[1]):
        df_train.iloc[i,j] = text_preprocessing_pipeline(df_train.iloc[i,j])
df_train.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,combined_news
0,2008/8/8,0,georgia downs two russian warplanes as countri...,breaking musharraf to be impeached,russia today columns of troops roll into south...,russian tanks are moving towards the capital o...,afghan children raped with impunity u n offici...,russian tanks have entered south ossetia whils...,breaking georgia invades south ossetia russia ...,the enemy combatent trials are nothing but a s...,...,al qaeda faces islamist backlash,condoleezza rice the us would not act to preve...,this is a busy day the european union has appr...,georgia will withdraw soldiers from iraq to he...,why the pentagon thinks attacking iran is a ba...,caucasus in crisis georgia invades south ossetia,indian shoe manufactory and again in a series ...,visitors suffering from mental illnesses banne...,no help for mexico s kidnapping surge,georgia downs two russian warplanes as countri...
1,2008/8/11,1,why wont america and nato help us if they wont...,bush puts foot down on georgian conflict,jewish georgian minister thanks to israeli tra...,georgian army flees in disarray as russians ad...,olympic opening ceremony fireworks faked,what were the mossad with fraudulent new zeala...,russia angered by israeli military sale to geo...,an american citizen living in s ossetia blames...,...,do not believe tv neither russian nor georgian...,riots are still going on in montreal canada be...,china to overtake us as largest manufacturer,war in south ossetia,israeli physicians group condemns state torture,russia has just beaten the united states over ...,perhaps the question about the georgia russia ...,russia is so much better at war,so this is what it is come to trading sex for ...,why wont america and nato help us if they wont...
2,2008/8/12,0,remember that adorable year old who sang at th...,russia ends georgia operation,if we had no sexual harassment we would have n...,al qa eda is losing support in iraq because of...,ceasefire in georgia putin outmaneuvers the west,why microsoft and intel tried to kill the xo l...,stratfor the russo georgian war and the balanc...,i am trying to get a sense of this whole georg...,...,why russias response to georgia was right,gorbachev accuses america of making a serious ...,russia georgia and nato cold war two,remember that adorable year old who led your c...,war in georgia the israeli connection,all signs point to the us encouraging georgia ...,christopher king argues that the us and nato a...,america the new mexico,bbc news asia pacific extinction by man not cl...,remember that adorable year old who sang at th...
3,2008/8/13,0,america refuses israel weapons to attack iran ...,when the president ordered to attack tskhinval...,israel clears troops who killed reuters cameraman,britain s policy of being tough on drugs is po...,body of year old found in trunk latest ransom ...,china has moved million quake survivors into p...,bush announces operation get all up in russia ...,russian forces sink georgian ships,...,us humanitarian missions soon in georgia if ru...,georgia s ddos came from us sources,russian convoy heads into georgia violating truce,israeli defence minister us against strike on ...,gorbachev we had no choice,witness russian forces head towards tbilisi in...,quarter of russians blame america for conflict...,georgian president says us military will take ...,nobel laureate aleksander solzhenitsyn accuses...,america refuses israel weapons to attack iran ...
4,2008/8/14,1,all the experts admit that we should legalise ...,war in south osetia pictures made by a russian...,swedish wrestler ara abrahamian throws away me...,russia exaggerated the death toll in south oss...,missile that killed inside pakistan may have b...,rushdie condemns random house s refusal to pub...,poland and us agree to missle defense deal int...,will the russians conquer tblisi bet on it no ...,...,georgia confict could set back russia

In [10]:
for i in range(df_valid.shape[0]):
    for j in range(2, df_valid.shape[1]):
        df_valid.iloc[i,j] = text_preprocessing_pipeline(df_valid.iloc[i,j])
df_valid.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,combined_news
0,2014/12/3,1,british man found with nailbomb weapons cache ...,israel must give up nuclear weapons united nat...,ukraine pm discloses accident at nuclear plant...,russian state tv s latest attempt to persuade ...,dna confirms king richard iii s remains under ...,an indian state has halted mass sterilisations...,north korea bans name jong un for all but lead...,president bashar al assad of syria says us led...,...,israel vents their fury with france after parl...,indian forces kill militants crossing into kas...,melting rate of west antarctica glaciers tripl...,green energy sector jobs surpass total oil san...,jordanian muslim brotherhood smuggling weapons...,questions raised over queens ancestry after dn...,topless femen activist goes on trial in cologn...,west antarctic melt rate has tripled the total...,nasa has announced that a test launch of their...,british man found with nailbomb weapons cache ...
1,2014/12/4,0,nasa officially announces manned mars mission ...,vatican finds hundreds of millions of euros tu...,major cruise lines released over a billion gal...,isis recruits are begging to be allowed to ret...,angela merkel speaks against net neutrality,president vladimir putin warned russians of ha...,iran sentences blogger to death for insulting ...,isis claims constructing dirty bomb after stea...,...,worlds biggest cave in threatened by cable car...,south korea will build a giant christmas tree ...,u k introduces the google tax the government t...,cement intended for gaza reconstruction ends u...,a japanese artist who made a kayak modelled on...,putin vows to punish speculators pushing down ...,nasa s orion first test flight for manned mars...,iran has provided evidence to the u n atomic a...,life under isis rule a living hell mosul iraq ...,nasa officially announces manned mars mission ...
2,2014/12/5,1,orion deep space capsule launches,dozens of chinese held in kenya accused of pre...,germany tries year old man who fought with isi...,blame your boss not china for stagnant wage gr...,france ordered to pay somali pirates up to eac...,chinese researchers report that a common pantr...,china overtakes the us as world s largest economy,tony blair says politicians are underpaid,...,corruption a problem for all latest global ind...,islamic militants attack grozny chechnyas capi...,the new m alan turing institute for data scien...,cameroons national assembly has voted into law...,u k to repay world war i loan the war loan a s...,uk mass surveillance laws do not breach human ...,latvian politician forced to step down after t...,reports obama mulling sanctions on israel,putin tells russia hard times are coming,orion deep space capsule launches dozens of ch...
3,2014/12/8,0,mass surveillance exposed by snowden not justi...,the body temperature of polish toddler rescued...,mexico says evidence proves missing students w...,sixteen british police officers have been char...,the best known liberal tv channel in russia ha...,uber banned in india s capital city new delhi ...,new horizons robotic probe awakened begins exp...,uber the america online ride hailing service h...,...,eu parliament calls on pakistan to abolish the...,turkish president erdogan says that the new pr...,in canada more jobs in green energy than tar s...,poland has long seen itself as stuck between t...,rome mayor defies mobster threats and refuses ...,german chancellor angela merkel accuses russia...,arrested for looting antiquities from israel s...,ukraine to allow russian electricity imports a...,bc poultry farms have been placed under quaran...,mass surveillance exposed by snowden not justi...
4,2014/12/9,0,hackers tell sony to halt the release of the i...,elephant slaughter ivory sales out of control ...,germany thousands join anti islam protest agai...,nsa warrantless bulk phone metadata spying con...,hong kong protesters given thursday d

In [11]:

for i in range(df_test.shape[0]):
    for j in range(2, df_test.shape[1]):
        df_test.iloc[i,j] = text_preprocessing_pipeline(df_test.iloc[i,j])
df_test.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,combined_news
0,2015/9/18,0,brazil s supreme court has banned corporate co...,investigation finds exxon knew about co s effe...,islamic extremist shot dead in berlin after st...,man sentenced to death in saudi arabia will be...,finland has a strike over people have gathered...,cameron gives top environment policy job to oi...,india is not an it superpower but an it labour...,bid to inspect israeli nuclear site blocked,...,one million yemenis ready to invade saudi arab...,israeli police could use live sniper fire agai...,japan passes new military bill scuffling ensues,russia says ready for talks with united states...,farmers in northern france have been ordered t...,super gonorrhoea outbreak in leeds,in egalitarian sweden richer regions reluctant...,china is building the mother of all reputation...,half a million children have fled attacks by t...,brazil s supreme court has banned corporate co...
1,2015/9/21,1,volkswagen has admitted to using software on v...,saudi teenager ali mohammed al nimr has been s...,german woman is charged with counts of accesso...,new zealand zoo will not euthanize tiger that ...,hungary passes law allowing govt to use army i...,hungary places ads in lebanese press warning o...,ruble fall causes wage parity between russia a...,the obama administration will increase the num...,...,isis defectors reveal growing disillusionment,ministry of defence condemns serving army gene...,apple s app store infected with malware in china,islamic rebels free american hostages held in ...,israel to bring in chinese construction workers,reports russian troops refusing deployment to ...,at least people killed by three bomb blasts in...,climate change will boost arctic mosquito popu...,greece opposition concedes defeat syriza poise...,volkswagen has admitted to using software on v...
2,2015/9/22,0,another drug cycloserine sees a price jump ove...,hedge fund trader martin shkreli who raised th...,american soldiers told to ignore sexual abuse ...,an engineer who once worked for russia s milit...,vw emissions scandal hits m vehicles world wide,israeli police can now use snipers against tee...,fewer than one in lobbyists are covered by the...,seeds removed from doomsday seed vault as a re...,...,facebook accused of spying on belgian citizens...,german stocks fall more than pulled down by vo...,midwife in the indian state of assam registere...,exxon confirmed global warming consensus in wi...,syrian asylum seeker in germany filmed boastin...,syrian refugee tripped over by hungarian journ...,land degradation desertification might create ...,european union officials will forge ahead with...,german police conduct raids in berlin s islami...,another drug cycloserine sees a price jump ove...
3,2015/9/23,0,pharmaceuticals ceo who raised hiv drug price ...,norway rewards brazil with billion for keeping...,volkswagen hires bp oil spill lawyers to defen...,turing to roll back pricing for daraprim amid ...,ginger extremist convicted in royal death plot...,martin winterkorn resigns as volkswagen ceo,pope francis to dine with homeless rather than...,this year old won the google science fair with...,...,putin opens the biggest mosque in russia prais...,world s largest d mud printer makes dirt cheap...,hillary clinton comes out against keystone xl ...,the israeli secret service is suspected of bei...,canadians eating less meat taking a bite out o...,david cameron says europe must get better at s...,eight million tons of plastic dumped in ocean ...,a civil lawsuit against volkswagen was filed t...,diesel worse than the black plague and respons...,pharmaceuticals ceo who raised hiv drug price ...
4,2015/9/24,0,canadian mining company barrick gold leaks a m...,nigerian troops free women children in boko ha...,hajj dead and injured in mina stampede,the associate press will no longer use the ter...,german commander in charge of anti isis

In [12]:
max_len = 0
for i in range(df_train.shape[0]):
  leng = len(df_train.iloc[i, -1].split())
  if max_len < leng:
    max_len = leng
print(max_len)

724


In [13]:
def stopwords_removal(news):
  word_list = news.split()
  tmp = []
  for word in word_list:
    if word.lower() not in stopwords.words('english'):
      tmp.append(word)
    filtered_news_article = ' '.join(tmp)
    return filtered_news_article

In [14]:
train_data = (df_train['combined_news'].to_list(), df_train['Label'].to_list()) 
#train_data = (df_train['new_combined_news'].to_list(), df_train['Label'].to_list()) 

In [15]:
test_data = (df_test['combined_news'].to_list(), df_test['Label'].to_list())
#test_data = (df_test['new_combined_news'].to_list(), df_test['Label'].to_list())

In [16]:
def create_dataset(texts: List[str],labels: List[int],num_classes: int = 2,class_names: List[int] = [0, 1]) -> Dataset:
    """
    Create HuggingFace Dataset from texts and labels.
    """
    ds = Dataset.from_dict({'text': texts, 'label': labels})
    ds = ds.cast_column('label', ClassLabel(num_classes=num_classes, names=class_names))
    return ds

In [17]:
def fine_tune(
    train_data: Tuple[List[str], List[int]], # text, label
    eval_data: Tuple[List[str], List[int]],
    model_output_dir: Union[str, Path],
    model_name: str = "distilbert-base-uncased",
    num_classes: int = 2,
    class_names: List[int] = [0, 1],
    learning_rate: float = 1e-5,
    weight_decay: float = 0.3,
    steps: int = 50,
    batch_size: int = 16,
    epochs: int = 20,
    verbose: bool = True
) -> Trainer:
    """
    Fine-tune a transformer for text classification.
    """

    def model_init():
        return AutoModelForSequenceClassification.from_pretrained(
            model_name, num_labels=num_classes
        )
    
    # create datasets
    if verbose:
        print('Creating train and evaluation datasets.')
    train_ds = create_dataset(train_data[0], train_data[1], num_classes, class_names)
    eval_ds = create_dataset(eval_data[0], eval_data[1], num_classes, class_names)

    # perform shuffling
    train_ds = train_ds.shuffle(seed=42)
    eval_ds = eval_ds.shuffle(seed=42)

    if verbose:
        print(f'Tokenizing inputs.')
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenize = lambda eg: tokenizer(eg['text'], truncation=True)
    tokenized_train = train_ds.map(tokenize, batched=True)
    tokenized_eval = eval_ds.map(tokenize, batched=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # define training arguments
    training_args = TrainingArguments(
        output_dir=model_output_dir,
        evaluation_strategy='steps',#steps or epoch
        # save_strategy = 'epoch',
        eval_steps=steps,
        logging_strategy='steps',
        logging_steps=steps,
        #saving_strategy='steps',
        #saving_steps=steps,
        save_strategy='steps',
        save_steps=steps,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs, 
        load_best_model_at_end=True,
    )

    # perform fine-tuning
    if verbose:
        print("Fine-tuning model.")
    trainer = Trainer(
        model_init=model_init, 
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        tokenizer=tokenizer,
        data_collator=data_collator,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5, early_stopping_threshold=0.0)]
    )
    trainer.train()
    # save model
    if verbose:
        print(f'Saving best model to {model_output_dir}/best-model.')
    trainer.save_model(f'{model_output_dir}/best-model')
    return trainer


In [18]:
def predict(trainer: Trainer, texts: List[str]) -> Tuple[List[int], List[float]]:
    """
    Return predicted class label and probabilities.
    """

    # create dataset and tokenize
    ds = Dataset.from_dict({'text': texts})
    tokenize = lambda eg: trainer.tokenizer(eg['text'], truncation=True, padding=True)
    tokenized_ds = ds.map(tokenize, batched=True)
    # make predictions
    logits = trainer.predict(tokenized_ds).predictions
    softmax = scipy.special.softmax(logits, axis=-1)
    prediction = softmax.argmax(axis=-1).tolist()
    probabilities = softmax.tolist()
    return prediction, probabilities

In [19]:
def evaluate(y: List[int], y_pred: List[int], class_names: List[int] = [0, 1], verbose=True):
    """
    Return evaluation metrics and print the confusion matrix.
    """

    # generate confusion matrix
    cf_mat = metrics.confusion_matrix(y, y_pred)
    row_names = ['Actual ' + str(c) for c in class_names]
    col_names = ['Predicted ' + str(c) for c in class_names]
    cf_mat = pd.DataFrame(cf_mat, index=row_names, columns=col_names)

    # calculate performance metrics
    accuracy = metrics.accuracy_score(y, y_pred)
    precision = metrics.precision_score(y, y_pred, average=None)
    recall = metrics.recall_score(y, y_pred, average=None)
    f1 = metrics.f1_score(y, y_pred, average=None)

    output_dict = {}
    output_dict['Accuracy'] = accuracy
    output_dict['Precision'] = precision
    output_dict['Recall'] = recall
    output_dict['F1'] = f1

    #print("Debug1:") ###
    if verbose:
        print(cf_mat)
        for k, v in output_dict.items():
            #print("Debug2:") ###
            print(k, ': ', v)
    
    return output_dict

In [20]:
def fine_tune_pipeline(
    train_data: Tuple[List[str], List[int]], # text, label
    eval_data: Tuple[List[str], List[int]],
    model_output_dir: Union[str, Path],
    model_name: str = "distilbert-base-uncased",
    num_classes: int = 2,
    class_names: List[int] = [0, 1],
    learning_rate: float = 1e-5,
    weight_decay: float = 0.3,
    steps: int = 50,
    batch_size: int = 16,
    epochs: int = 20,
    verbose: bool = True
) -> Dict[str, Any]:
    """
    Pipeline for fine-tuning a transformer and evaluating the performance.
    """

    # fine-tuning
    trainer = fine_tune(
        train_data,
        eval_data,
        model_output_dir,
        model_name,
        num_classes,
        class_names,
        learning_rate,
        weight_decay,
        steps,
        batch_size,
        epochs,
        verbose
    )

    # predict and evaluate on train & validation data
    train_pred, train_proba = predict(trainer, train_data[0])
    eval_pred, eval_proba = predict(trainer, eval_data[0])

    if verbose:
        print('Performance of tuned model on training set:')
    train_metrics = evaluate(train_data[1], train_pred, class_names)
    print()
    if verbose:
        print('Performance of tuned model on evaluation set:')
    eval_metrics = evaluate(eval_data[1], eval_pred, class_names)

    output = {
        'train_prediction': train_pred,
        'train_probabilities': train_proba,
        'train_performance': train_metrics,
        'eval_prediction': eval_pred,
        'eval_probabilities': eval_proba,
        'eval_performance': eval_metrics,
    }

    return output

In [21]:
save_model_dir = "/content/drive/MyDrive/5246Project/save_weights"
output = fine_tune_pipeline(train_data=train_data, eval_data=test_data, model_output_dir=save_model_dir, learning_rate=5e-6)

Creating train and evaluation datasets.


Casting the dataset:   0%|          | 0/1591 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/199 [00:00<?, ? examples/s]

Tokenizing inputs.


Map:   0%|          | 0/1591 [00:00<?, ? examples/s]

...

Step,Training Loss,Validation Loss
50,0.690800,0.692039
100,0.694400,0.690167
150,0.694300,0.689977
200,0.685500,0.691844
250,0.688400,0.690130
300,0.689200,0.689732
350,0.686900,0.690004
400,0.682600,0.689175
450,0.673800,0.705765
500,0.676800,0.693206


Saving best model to /content/drive/MyDrive/5246Project/save_weights/best-model.


Map:   0%|          | 0/1591 [00:00<?, ? examples/s]

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

Performance of tuned model on training set:
          Predicted 0  Predicted 1
Actual 0           19          709
Actual 1            0          863
Accuracy :  0.5543683218101823
Precision :  [1.         0.54898219]
Recall :  [0.0260989 1.       ]
F1 :  [0.05087015 0.70882957]

Performance of tuned model on evaluation set:
          Predicted 0  Predicted 1
Actual 0            0           92
Actual 1            1          106
Accuracy :  0.5326633165829145
Precision :  [0.         0.53535354]
Recall :  [0.         0.99065421]
F1 :  [0.         0.69508197]
